In [ ]:
from transformers import BartForConditionalGeneration,BartTokenizer
from transformers import AutoTokenizer,AutoModelForSequenceClassification,AdamW

import torch


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("model/bert-base-uncased/")
model =AutoModelForSequenceClassification .from_pretrained("model/bert-base-uncased/")

In [ ]:
sequences = ["Using a Transformer network is simple","i am a super hero"]

In [ ]:
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")


batch["labels"] = torch.tensor([1, 1])     #字典里添加一个labels

optimizer = AdamW(model.parameters())    #adam优化器
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [ ]:
batch

In [ ]:
loss

In [ ]:
optimizer

In [ ]:
from datasets import load_dataset,list_datasets
datasets_list = list_datasets()
len(datasets_list)

In [ ]:
datasets = load_dataset('squad')
print(datasets)

In [ ]:
from datasets import load_dataset
dataset=load_dataset("glue", "mrpc")

dataset.save_to_disk("dataset\mrpc")

In [ ]:
dataset

In [ ]:
from datasets import load_from_disk
dataset_of_disk=load_from_disk("dataset\mrpc")

In [ ]:
dataset_of_disk

In [ ]:
raw_train_dataset = dataset_of_disk["train"]
#len(raw_train_dataset)

raw_train_dataset[4]

In [ ]:
raw_train_dataset.features

In [ ]:
from transformers import AutoTokenizer

In [ ]:
dataset_of_disk["train"]["sentence1"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('model/bert-base-uncased/')
tokenized_sentences_1 = tokenizer(dataset_of_disk["train"]["sentence1"])         #tokenizer train全部的sentence1
tokenized_sentences_2 = tokenizer(dataset_of_disk["train"]["sentence2"])         #tokenizer train全部的sentence2

In [ ]:
tokenized_sentences_2

以下两种结果不一样，一个是两个句子分开的情况，一个是相关联的列表

In [ ]:
#inputs = tokenizer("This is the first sentence.", "This is the second one.",return_tensors="pt")
inputs = tokenizer("This is the first sentence.", "This is the second one two three four.",padding='max_length',max_length=20)
inputs

In [ ]:
inputs = tokenizer(["This is the first sentence.", "This is the second one two three four."])
inputs

In [ ]:
sentence1=['This is the first sentence.','This is the second sentence.','This is the third sentence.','This is the four sentence.']

sentence2=['This is the first sentences.','This is the second sentences.','This is the third sentences.','This is the four sentences one two three.',]


In [ ]:
inputs = tokenizer(sentence1,sentence2,truncation=True)
inputs

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][1])

In [ ]:
inputs = tokenizer(["This is the first sentence.","one two three four five.","one two three four five."], ["This is the second one two three four.","six seven egiht nine.","one two three four five."],padding=True,truncation=True)
inputs


In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

得到的input_id和原文似乎有出入,decode看下

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][1])

In [ ]:

tokenized_dataset = tokenizer(
    dataset_of_disk["train"]["sentence1"],
    dataset_of_disk["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = dataset_of_disk.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
tokenized_datasets['train'][0]['attention_mask']

动态padding

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_datasets["train"][:8]
samples

In [ ]:
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

取出每个句子的input_id的长度，动态padding是要都padding到batch的最大值

In [ ]:
batch = data_collator(samples)       #1行代码

In [ ]:
{k: v.shape for k, v in batch.items()}  #key和value

train

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_from_disk

In [ ]:
raw_dataset=load_from_disk("dataset\mrpc")
checkpoint = "model/bert-base-uncased/"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


In [ ]:
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")      #唯一的参数是新模型保存路径

In [ ]:
training_args

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()